In [1]:
import pandas as pd;
import datetime as dt;
import numpy as np;
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import scale
import pickle

In [13]:
weather = pd.read_csv("weather (table 7)_training_update.csv")
trajectories = pd.read_csv("trajectories(table 5)_training.csv")

#trajectories

del trajectories['vehicle_id']
del trajectories['travel_seq']

trajectories['starting_time'] =  pd.to_datetime(trajectories['starting_time'], format = "%Y-%m-%d %H:%M:%S")

st = []

for c in trajectories['starting_time']:
    st.append(c - dt.timedelta(seconds = c.second, minutes = c.minute))
    
trajectories['starting_time'] = st

trajectories = trajectories.groupby(['intersection_id', 'tollgate_id', 'starting_time']).mean()

trajectories = trajectories.reset_index()

datehour = []

for c in trajectories['starting_time']:
    discard = dt.timedelta(hours = c.hour % 3, minutes = c.minute, seconds = c.second)
    if (discard > dt.timedelta(hours = 1, minutes = 30)):
        c = c + dt.timedelta(hours = 3)
    datehour.append(c - discard)
    
trajectories['datehour'] = datehour

#weahter

weather['datehour'] = pd.to_timedelta(weather['hour'], unit="H") + pd.to_datetime(weather['date'], format="%Y-%m-%d")

del weather['hour']
del weather['date']

#merge
training = pd.merge(trajectories, weather, on = 'datehour')

del training['datehour']

training = training.sort_values(by = ['intersection_id', 'tollgate_id', 'starting_time'])

M = []
H = []
w = []

for c in training['starting_time']:
    w.append(int(dt.date.strftime(c, format="%w")));
    H.append(c.hour)
    M.append(c.minute)
    
training['hour'] = H
training['week'] = w
#training['minute'] = M

training.loc[training['week']==0, 'week'] = 7
training.loc[training['intersection_id']=='A', 'intersection_id'] = 1
training.loc[training['intersection_id']=='B', 'intersection_id'] = 2
training.loc[training['intersection_id']=='C', 'intersection_id'] = 3

del training['starting_time']

cols = training.columns

tmp = cols[:cols.get_loc('travel_time')];
tmp = tmp.append(cols[cols.get_loc('travel_time') + 1:])
tmp = tmp.append(cols[cols.get_loc('travel_time'):(cols.get_loc('travel_time')+1)])

training.dropna()

training = training[tmp]

training.to_csv("training.csv")


In [5]:
import re

submission = pd.read_csv("submission_sample_travelTime.csv")
testing_weather = pd.read_csv("weather (table 7)_test1.csv")

#submission
datetime = []

for c in submission['time_window']:
    c = c.translate(None, '[)')
    datetime.append(c.split(',')[0])
    
submission['datetime'] = datetime;
submission['datetime'] =  pd.to_datetime(submission['datetime'], format = "%Y-%m-%d %H:%M:%S")

datehour = []

for c in submission['datetime']:
    discard = dt.timedelta(hours = c.hour % 3, minutes = c.minute, seconds = c.second)
    if (discard > dt.timedelta(hours = 1, minutes = 30)):
        c = c + dt.timedelta(hours = 3)
    datehour.append(c - discard)
    
submission['datehour'] = datehour

#weahter

testing_weather['datehour'] = pd.to_timedelta(testing_weather['hour'], unit="H") + pd.to_datetime(testing_weather['date'], format="%Y-%m-%d")

del testing_weather['hour']
del testing_weather['date']

#merge
submission = pd.merge(submission, testing_weather, on = 'datehour')

submission.to_csv('submission_raw.csv')

del submission['datehour']

M = []
H = []
w = []

for c in submission['datetime']:
    w.append(int(dt.date.strftime(c, format="%w")));
    H.append(c.hour)
    M.append(c.minute)
    
submission['hour'] = H
submission['week'] = w
submission['minute'] = M


submission.loc[submission['week']==0, 'week'] = 7
submission.loc[submission['intersection_id']=='A', 'intersection_id'] = 1
submission.loc[submission['intersection_id']=='B', 'intersection_id'] = 2
submission.loc[submission['intersection_id']=='C', 'intersection_id'] = 3

del submission['datetime']

cols = submission.columns

tmp = cols[:cols.get_loc('avg_travel_time')];
tmp = tmp.append(cols[cols.get_loc('avg_travel_time') + 1:])
tmp = tmp.append(cols[cols.get_loc('avg_travel_time'):(cols.get_loc('avg_travel_time')+1)])

submission.dropna()

submission = submission[tmp]

#del submission['time_window']

submission.to_csv("submission.csv")


In [42]:
weather = pd.read_csv("weather (table 7)_test1.csv")
trajectories = pd.read_csv("trajectories(table 5)_test1.csv")

#trajectories

del trajectories['vehicle_id']
del trajectories['travel_seq']

trajectories['starting_time'] =  pd.to_datetime(trajectories['starting_time'], format = "%Y-%m-%d %H:%M:%S")

st = []

for c in trajectories['starting_time']:
    st.append(c - dt.timedelta(seconds = c.second, minutes = c.minute % 20))
    
trajectories['starting_time'] = st

trajectories = trajectories.groupby(['intersection_id', 'tollgate_id', 'starting_time']).mean()

trajectories = trajectories.reset_index()

datehour = []

for c in trajectories['starting_time']:
    discard = dt.timedelta(hours = c.hour % 3, minutes = c.minute, seconds = c.second)
    if (discard > dt.timedelta(hours = 1, minutes = 30)):
        c = c + dt.timedelta(hours = 3)
    datehour.append(c - discard)
    
trajectories['datehour'] = datehour

#weahter

weather['datehour'] = pd.to_timedelta(weather['hour'], unit="H") + pd.to_datetime(weather['date'], format="%Y-%m-%d")

del weather['hour']
del weather['date']

#merge
testing = pd.merge(trajectories, weather, on = 'datehour')

del testing['datehour']

testing = testing.sort_values(by = ['intersection_id', 'tollgate_id', 'starting_time'])

M = []
H = []
w = []

for c in testing['starting_time']:
    w.append(int(dt.date.strftime(c, format="%w")));
    H.append(c.hour)
    M.append(c.minute)
    
testing['hour'] = H
testing['week'] = w
testing['minute'] = M

testing.loc[testing['week']==0, 'week'] = 7
testing.loc[testing['intersection_id']=='A', 'intersection_id'] = 1
testing.loc[testing['intersection_id']=='B', 'intersection_id'] = 2
testing.loc[testing['intersection_id']=='C', 'intersection_id'] = 3

del testing['starting_time']

cols = testing.columns

tmp = cols[:cols.get_loc('travel_time')];
tmp = tmp.append(cols[cols.get_loc('travel_time') + 1:])
tmp = tmp.append(cols[cols.get_loc('travel_time'):(cols.get_loc('travel_time')+1)])

testing.dropna()

testing = testing[tmp]

testing.to_csv("testing.csv")

In [36]:
testing

,intersection_id,tollgate_id,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,hour,week,minute,travel_time
0,1,2,1012.5,1017.5,128.0,1.6,23.4,73.0,0.0,6,2,0,41.097143
1,1,2,1012.5,1017.5,128.0,1.6,23.4,73.0,0.0,6,2,20,43.681667
2,1,2,1012.5,1017.5,128.0,1.6,23.4,73.0,0.0,6,2,40,68.024286
3,1,2,1012.5,1017.5,128.0,1.6,23.4,73.0,0.0,7,2,0,52.608333
4,1,2,1012.5,1017.5,128.0,1.6,23.4,73.0,0.0,7,2,20,56.165556
27,1,2,1013.0,1018.0,205.0,0.6,20.9,94.0,1.8,7,2,40,63.598571
32,1,2,1013.7,1018.7,90.0,1.9,20.1,95.0,0.1,15,2,0,99.960667
33,1,2,1013.7,1018.7,90.0,1.9,20.1,95.0,0.1,15,2,20,88.360556
34,1,2,1013.7,1018.7,90.0,1.9,20.1,95.0,0.1,15,2,40,74.886000
35,1,2,1013.7,1018.7,90.0,1.9,20.1,95.0,0.1,16,2,0,63.302727


In [40]:
training

,intersection_id,tollgate_id,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,hour,week,minute,travel_time
0,1,2,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0,0,2,20,58.050000
1,1,2,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0,1,2,20,56.870000
10,1,2,1000.5,1005.3,212.0,3.8,31.7,65.0,0.0,1,2,40,77.740000
11,1,2,1000.5,1005.3,212.0,3.8,31.7,65.0,0.0,2,2,20,42.640000
12,1,2,1000.5,1005.3,212.0,3.8,31.7,65.0,0.0,2,2,40,40.173333
13,1,2,1000.5,1005.3,212.0,3.8,31.7,65.0,0.0,3,2,20,41.920000
14,1,2,1000.5,1005.3,212.0,3.8,31.7,65.0,0.0,3,2,40,39.430000
15,1,2,1000.5,1005.3,212.0,3.8,31.7,65.0,0.0,4,2,0,48.130000
16,1,2,1000.5,1005.3,212.0,3.8,31.7,65.0,0.0,4,2,20,62.106667
27,1,2,999.7,1004.5,239.0,1.9,31.8,64.0,0.0,4,2,40,46.115000


In [6]:
submission

,intersection_id,tollgate_id,time_window,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,hour,week,minute,avg_travel_time
0,1,2,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",1013.0,1018.0,205.0,0.6,20.9,94.0,1.8,8,2,0,681.30
1,1,2,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",1013.0,1018.0,205.0,0.6,20.9,94.0,1.8,8,2,20,128.11
2,1,2,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",1013.0,1018.0,205.0,0.6,20.9,94.0,1.8,8,2,40,176.23
3,1,2,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",1013.0,1018.0,205.0,0.6,20.9,94.0,1.8,9,2,0,749.09
4,1,2,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",1013.0,1018.0,205.0,0.6,20.9,94.0,1.8,9,2,20,37.22
5,1,2,"[2016-10-18 09:40:00,2016-10-18 10:00:00)",1013.0,1018.0,205.0,0.6,20.9,94.0,1.8,9,2,40,184.95
6,1,3,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",1013.0,1018.0,205.0,0.6,20.9,94.0,1.8,8,2,0,390.30
7,1,3,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",1013.0,1018.0,205.0,0.6,20.9,94.0,1.8,8,2,20,581.84
8,1,3,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",1013.0,1018.0,205.0,0.6,20.9,94.0,1.8,8,2,40,35.58
9,1,3,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",1013.0,1018.0,205.0,0.6,20.9,94.0,1.8,9,2,0,636.40
